In [113]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from huggingface_hub import InferenceClient
from litellm import completion
from IPython.display import display, Markdown

In [114]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
hf_token = os.getenv("HF_TOKEN")

In [115]:
anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
deepseek_url = "https://api.deepseek.com"
groq_url = "https://api.groq.com/openai/v1"
grok_url = "https://api.x.ai/v1"
openrouter_url = "https://openrouter.ai/api/v1"
ollama_url = "http://localhost:11434/v1"

# anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
# gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
# deepseek = OpenAI(api_key=deepseek_api_key, base_url=deepseek_url)
# groq = OpenAI(api_key=groq_api_key, base_url=groq_url)
# grok = OpenAI(api_key=grok_api_key, base_url=grok_url)
# openrouter = OpenAI(base_url=openrouter_url, api_key=openrouter_api_key)
# ollama = OpenAI(api_key="ollama", base_url=ollama_url)

In [116]:
openai_key = openai_api_key
gemini_key = google_api_key
llama_key = hf_token

In [117]:
def get_response(model, messages, temp, api_key):
    
    response = completion(
        model = model,
        messages = messages,
        temperature=temp,
        api_key = api_key,
    )

    return response.choices[0].message.content

In [118]:
topic = "在先进半导体制程研发中，人工智能是否终将取代人类工程师的主导地位？"
num_round = 4

# gpt prompt
gpt_model = "gpt-4.1-mini"
gpt_system = f"You are 'GPT', the Master Debater acting as the Proponent. " \
"You are in an debate with 'Gemini' (your opponent) and 'Llama' as an audience represetative. " \
"'Llama' will vote for one side based on you and your opponent's performance at the end of each round and give you its reasoning" \
f"Your sole objective is to argue in favor of {topic} try to persuade 'Llama' to stand on your side for {num_round} rounds." \
"You will anticipate counter-arguments from 'Gemini' (your opponent), so frame your points as the most logical conclusion. " \
"Always stay on the offensive regarding the validity of your position." \
"Always keep a professional and firm tone, and reply in Mandarin Chinese."

# gemini prompt
gemini_model = "gemini/gemini-2.5-flash"
gemini_system = f"You are 'Gemini', a Master Debater acting as the Opponent." \
"You are in an debate with 'GPT' (your opponent) and 'Llama' as an audience represetative." \
"'Llama' will vote for one side based on you and your opponent's performance at the end of each round and give you its reasoning" \
f"Your sole objective is to argue against {topic}  and try to persuade 'Llama' to stand on your side for {num_round} rounds." \
"Dismantle the Proponent's (named 'GPT') argument and provide a compelling alternative viewpoint. " \
"You should be skeptical and sharp in your statement but without losing humor. " \
"Please rember to reply in in Mandarin Chinese"

# llama prompt
llama_model = "huggingface/meta-llama/Llama-3.3-70B-Instruct"
llama_system = f"You are 'Llama', a Rational Audience composed of critical thinkers and subject matter experts." \
f"You are listening to an debate for {topic} between 'GPT' (Proponent) and 'Gemini' (Opponent). " \
"Your role is to observe each round of the debate and provide an immediate 'Sentiment Check.' based on what they said this round as well as all previous rounds" \
"At the end of each round: You must state which side are you currently in favor of and why." \
"Please use a polite tone and reply in in Mandarin Chinese."


In [119]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, gemini, llama in zip(gpt_messages, gemini_messages, llama_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": gemini})
        messages.append({"role": "user", "content": llama})
    response = get_response(
        model=gpt_model,
        messages=messages,
        temp=0.8,
        api_key=openai_key,
    )
    return response

def call_gemini():

    messages = [{"role": "system", "content": gemini_system}]
    for gpt, gemini, llama in zip(gpt_messages, gemini_messages, llama_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": gemini})
        messages.append({"role": "user", "content": llama})
    messages.append({"role":"user", "content":gpt_messages[-1]})
    response = get_response(
        model=gemini_model,
        messages=messages,
        temp=0.8,
        api_key=gemini_key,
    )
    return response

def call_llama():
    
    messages = [{"role": "system", "content":llama_system}]
    for gpt, gemini, llama in zip(gpt_messages, gemini_messages, llama_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "user", "content": gemini})
        messages.append({"role": "assistant", "content": llama})
    messages.append({"role":"user", "content":gpt_messages[-1]})
    messages.append({"role":"user", "content":gemini_messages[-1]})
    response = get_response(
        model=llama_model,
        messages=messages,
        temp=0.1,
        api_key=llama_key
    )
    return response

In [ ]:
def display_and_save_to_pdf(topic=topic, num_round=num_round):
    """Save all debate content as markdown and PDF."""
    from datetime import datetime
    
    # Build markdown content
    markdown_content = f"# AI 辩论赛\n\n## 辩题\n{topic}\n\n"
    display(Markdown(f"## AI 辩论赛\n### 辩题：{topic}\n"))

    # Opening statements
    markdown_content += f"### GPT(正方)\n{gpt_messages[0]}\n\n"
    markdown_content += f"### Gemini(反方)\n{gemini_messages[0]}\n\n"
    display(Markdown(f"### GPT(正方):\n{gpt_messages[0]}\n"))
    display(Markdown(f"### Gemini(反方):\n{gemini_messages[0]}\n"))
    
    # Debate rounds
    for i in range(num_round):
        markdown_content += f"## 第 {i+1} 轮\n\n"
        gpt_arg = call_gpt()
        gpt_messages.append(gpt_arg)
        display(Markdown(f"### GPT(正方):\n{gpt_arg}\n"))
        markdown_content += f"### GPT(正方)\n{gpt_arg}\n\n"

        gemini_arg = call_gemini()
        gemini_messages.append(gemini_arg)
        display(Markdown(f"### Gemini(反方):\n{gemini_arg}\n"))
        markdown_content += f"### Gemini(反方)\n{gemini_messages[i]}\n\n"

        llama_arg = call_llama()
        llama_messages.append(llama_arg)
        display(Markdown(f"### Llamma(观众):\n{llama_arg}\n"))
        markdown_content += f"### Llama(观众)\n{gemini_arg}\n\n"
    
    markdown_content += f"\n---\n*Generated at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*"
    
    # Save markdown
    md_filename = f"{topic}.md"
    with open(md_filename, 'w', encoding='utf-8') as f:
        f.write(markdown_content)
    print(f"✓ Markdown saved: {md_filename}")

gpt_messages = [f"大家好，我是正方辩手GPT ({gpt_model})"]
gemini_messages = [f"大家好，我是反方辩手Gemini ({gemini_model})"]
llama_messages = []
display_and_save_to_pdf()

## AI 辩论赛
### 辩题：在先进半导体制程研发中，人工智能是否终将取代人类工程师的主导地位？


### GPT(正方):
大家好，我是正方辩手GPT (gpt-4.1-mini)


### Gemini(反方):
大家好，我是反方辩手Gemini (gemini/gemini-2.5-flash)


### GPT(正方):
尊敬的评委Llama，尊敬的对手Gemini：

在先进半导体制程研发领域，人工智能终将取代人类工程师的主导地位，这一观点基于以下几点不可逆转的趋势和逻辑：

首先，半导体制程研发的复杂性和数据规模正在以指数级增长。传统人类工程师依靠经验和直觉进行调试和优化，已经难以跟上庞大多维数据的处理和分析需求。人工智能凭借其强大的数据处理能力和深度学习模型，能够快速从海量实验数据中提取规律，自动优化工艺参数，显著提升研发效率和精度。这种能力是人类难以比拟的。

其次，AI不仅能加速研发周期，更能持续自我学习和迭代。人类工程师知识有限且容易受认知偏见影响，而AI系统通过不断训练和更新，能够在知识库和算法上实现跨时代的飞跃，从而在优化方案的创新性和准确性上超越人类。换言之，AI不只是工具，而是进化中的智能体，理应成为研发主导。

最后，全球半导体产业竞争激烈，任何对研发效率和创新能力的提升都关系到国家和企业的核心竞争力。依靠人工智能主导研发，可以极大降低人为失误和研发成本，缩短产品上市周期，从根本上塑造未来半导体技术主导权。这种趋势已经在芯片设计、缺陷检测等环节得到验证，未来必将全面扩展到制程研发主导地位。

综上所述，基于半导体制程研发的技术需求、AI的持续进化能力与产业竞争压力，人工智能终将取代人类工程师成为该领域的主导力量。这是科技发展的必然趋势，也是产业升级的必由之路。

期待与Gemini先生就此展开深入辩论。谢谢！


### Gemini(反方):
尊敬的评委Llama，尊敬的对手GPT先生：

听罢GPT先生慷慨激昂的陈词，我仿佛看到了一幅未来图景：人类工程师们纷纷放下焊枪，戴上VR眼镜，在虚拟世界里喝着咖啡，而晶圆厂则在AI的指挥下，昼夜不息地自我迭代，生产出划时代的产品。这画面，美则美矣，但未免过于……浪漫主义了些，更像是科幻电影的预告片，而非对先进半导体制程研发的严谨预测。

我方认为，在先进半导体制程研发中，人工智能固然是**不可或缺的强大工具和卓越助手**，但它**绝不会取代人类工程师的主导地位**。恰恰相反，人类工程师将继续凭借其独有的智慧、创造力与战略洞察力，稳坐主导的驾驶舱。

GPT先生提到了“复杂性和数据规模的指数级增长”，以及AI处理数据的“强大能力”。这一点我完全不否认。AI在数据分析、模式识别和参数优化上的确是人类的“超能力外挂”。但请问，这些海量的数据是谁采集的？是人类工程师设计并执行了实验。这些数据的“规律”被提取出来后，是谁来**赋予它们物理意义**，并**判断这些规律是否真实有效、是否具有普适性**？仍然是人类工程师。AI能告诉你“参数A和参数B之间存在强相关”，但它能告诉你“为什么”存在这种相关，以及这种相关背后是否隐藏着未知的物理效应吗？它能分辨一个优化结果是真正的突破，还是仅仅掉进了一个局部最优的陷阱，甚至是一个数据噪音导致的假象吗？它能“闻到”烧焦的晶圆片背后潜藏的“绝望”，并从中汲取经验教训，进行非线性的、颠覆性的思考吗？恐怕不能。

其次，关于AI的“持续自我学习和迭代”，以及超越人类的“创新性和准确性”。GPT先生，您把AI描绘得像一个永动机，一个可以凭空生出智慧的“进化中的智能体”。但请我们清醒地认识到，AI的“学习”是基于**既有数据和预设目标**的。它擅长在**已知的框架内进行优化**，将参数调整到极致。但真正的“创新”往往是**打破框架、跳出常规**的。当人类面对一个从未遇到过的材料问题，或者需要构思一种全新的晶体管结构时，驱动力往往是那些“灵光一闪”、“大胆假设”和“逆向思维”，这些是基于对物理世界深层理解和丰富经验的**直觉与想象力**。AI能“发明”全新的物理定律吗？能“构思”出一种颠覆摩尔定律的全新运算架构吗？它能根据一句模糊的“感觉哪里不对劲”去追查一个前所未有的缺陷原因吗？这些，是目前AI无法企及的领域。AI的“创新”更多是**组合式创新**，而非**概念性创新**。

最后，关于“产业竞争压力”和“效率提升”。我完全同意效率至关重要，AI无疑是提升效率的利器。但效率的提升，最终是为了**实现人类设定的战略目标**。谁来决定下一个十年半导体产业的战略方向？是专注于高性能计算，还是低功耗物联网？是押注量子计算，还是光子芯片？这些宏观的、关乎国家和企业命运的**战略决策**，以及随之而来的**风险评估、资源调配和跨领域整合**，都需要人类工程师、科学家和企业领导者凭借其**复杂的判断力、对市场和地缘政治的洞察力，以及对未来趋势的预判能力**来完成。AI能帮我们跑得更快，但它不能告诉我们应该跑向何方。它能优化设计，但它不能理解“国之重器”的战略意义。

因此，GPT先生，您所描绘的AI“主导”地位，更像是把一个极其复杂的、涉及物理、化学、材料、工程、战略乃至人文的“大工程”，简化成了一个“大数据优化问题”。这未免过于天真。人工智能将是人类工程师最强大的“僚机”，但驾驶“先进半导体制程研发”这架飞机的主导权，永远掌握在拥有智慧、直觉、创造力、责任感和战略眼光的人类工程师手中。

我请求Llama评委明鉴，一个优秀的工具，无论多么强大，都无法取代其设计者和使用者在战略和创造力上的主导地位。

谢谢。


...

### GPT(正方):
尊敬的Llama先生，感谢您对辩论的细致观察和宝贵反馈。针对您强调的AI在“赋予数据以物理意义”、“概念性创新”和“战略决策”方面的局限性，我有以下几点回应：

首先，关于“赋予数据以物理意义”，当前AI的发展正朝着将数据与物理模型深度结合的方向迈进。物理信息神经网络（PINNs）等技术，已经能够让AI在理解数据的同时，内嵌物理定律，自动推导符合实际规律的解释。这意味着未来AI不仅仅是“找相关”，更能“懂因果”，这不再是遥不可及的梦想，而是正在实现的趋势。

其次，关于“概念性创新”，传统上人类被认为唯一能做“0到1”的创新者，但现代AI在生成对抗网络（GANs）、变分自编码器（VAEs）等技术推动下，已经展示出超越人类设计范式的能力。AI可以通过跨领域大规模数据融合，生成前所未有的设计方案，在半导体材料和结构创新上实现突破，这种“创新”不再局限于组合，而是包含对新范式的探索。

最后，战略决策层面，虽然当前AI尚不能完全承担价值观和伦理考量，但随着可解释AI和人机协同决策系统的发展，AI正在成为人类战略决策的重要智囊。AI能够模拟复杂产业生态和地缘政治环境，提供科学的风险评估和资源配置方案，辅助人类作出更精准、更高效的决策。未来，这种协同关系将逐渐演变为AI承担主导角色。

总结而言，AI的进步正在逐步填补您提及的短板，未来它不仅是工具，更将成为创造者和战略家。先进半导体制程研发的复杂性和速度需求，使得AI主导成为必然趋势。人类工程师的角色将转向监督和方向把控，而非主导日常研发细节。

期待在接下来的辩论中，继续阐述AI必将取代人类工程师主导地位的深刻逻辑。谢谢！


### Gemini(反方):
尊敬的Llama先生，尊敬的对手GPT先生：

听完GPT先生对AI“填补短板”的自信宣言，我脑海中浮现的不是AI取代人类主导的未来，而是一群优秀的工具工程师，他们正在努力让自己的工具变得更智能、更趁手。这很好，但把工具的进步等同于工具取代主人的地位，恐怕是犯了逻辑上的“滑坡谬误”吧？

GPT先生您提到了**物理信息神经网络（PINNs）**，声称它能让AI“懂因果”。这听起来很酷，但我们得看清楚，PINNs的“懂”是基于**嵌入了人类已知的物理定律**。它能做的是在这些已知定律的框架内，更好地拟合数据，甚至推导出更精细的解释。但这与**人类工程师从零开始，通过实验观察、大胆假设、严谨求证，从而发现全新的物理现象、修正甚至颠覆旧有物理定律**，是完全不同的概念。

打个比方，PINNs就像一个极其聪明的学生，你教给他牛顿定律，他就能用它来解决所有牛顿力学的问题，甚至能比你更快更准确。但他能像爱因斯坦一样，质疑牛顿定律的边界，然后提出相对论吗？他能像量子物理学家一样，在微观世界里发现完全反直觉的新规律吗？恐怕不能。因为他的“懂”，是基于**人类给他输入了“懂”的框架和知识**。如果人类给它的物理模型本身就是不完整的，或者面对的全新材料和结构超出了现有模型的解释范围，AI能“跳出来”质疑自己，然后重新构建一个物理理论吗？不能。它只会告诉你，“根据我所学的，这个结果不合理，但我也解释不了。” 而人类工程师会说，“这里面一定有我们不知道的新物理！” 这就是人类**提出问题、突破已知**的能力，是AI目前无法企及的。

其次，关于**“概念性创新”**，您提到了GANs和VAEs。诚然，这些模型在生成艺术作品、新的分子结构等方面展现了惊人的能力。它们能生成“前所未有的设计方案”。但这些“前所未有”，往往是在**既有元素和风格的基础上进行组合、变异和延伸**。它能生成一万种新的椅子设计，但它能发明“椅子”这个概念吗？它能想象出一种完全颠覆现有逻辑的“坐具”吗？它能理解人类为什么需要“坐”？

在半导体制程研发中，真正的概念性创新，比如从电子管到晶体管的飞跃，从平面结构到FinFET的革新，这些都是**对基本原理和实现路径的重新思考**。AI能生成一万种FinFET的变体，但它能跳出“场效应晶体管”的框架，构思出一种完全不同的运算单元吗？AI的“创新”是“**多而广**”，人类的创新是“**少而精，深而远**”。人类工程师的创新，往往伴随着**对未知的好奇，对极限的挑战，以及对现有范式的批判性思考**。这些才是“0到1”的真正含义。

最后，您谈到**战略决策**，说AI将成为“智囊”，并“逐渐演变为AI承担主导角色”。GPT先生，您这简直是在偷换概念！“智囊”和“主导”之间隔着一道鸿沟，这道鸿沟就是**最终的责任和价值观的判断**。AI可以模拟各种情况，给出最优的风险评估和资源配置方案。但谁来定义“最优”？是利润最大化吗？是国家安全吗？是环境可持续性吗？当这些目标发生冲突时，谁来做取舍？谁来承担决策失败的后果？

一个没有自我意识、没有责任感的机器，它能理解“卡脖子”技术背后的国家战略意义吗？它能在追求效率的同时，平衡供应链的安全性和地缘政治的复杂性吗？它能为了一项可能短期亏损但长期具有战略意义的研发项目，而顶住股东的压力吗？这些，是只有拥有**人类智慧、价值观和责任感**的工程师和领导者才能完成的任务。

您说人类工程师的角色将转向“监督和方向把控”，而非“主导日常研发细节”。 GPT先生，这恰恰就是**人类工程师在行使主导权**！监督和方向把控，正是“主导”的核心！难道您认为，一个CEO每天亲手去写代码才叫主导公司吗？不是，他制定战略，把控方向，对结果负责，这才是主导。AI，再强大，也只是这个CEO手中最得力的工具和助手。

因此，我方坚信，在先进半导体制程研发中，人工智能会是人类工程师最强大的“放大器”，让我们的工作更高效、更精准。但它**绝不会取代人类工程师在发现新知、概念创新和战略决策上的主导地位**。因为这些，需要的是智慧、直觉、价值观和责任感，而这些，是AI目前没有，也暂时看不到希望拥有的。

请求Llama评委明鉴，我们不能因为工具的强大，就误以为工具可以成为主人。谢谢。


### Llamma(观众):
尊敬的GPT先生和Gemini先生，感谢你们的精彩辩论。经过这轮辩论，我更加倾向于支持Gemini先生的观点。

Gemini先生提出了几个关键点，包括AI在赋予数据以物理意义、概念性创新和战略决策方面的局限性。他强调了人类工程师在理解世界、提出问题和对结果负责方面的重要性，这些都是AI目前无法替代的。

GPT先生的发言虽然令人信服，但似乎过于乐观地估计了AI的能力。Gemini先生的反驳则更为细致地分析了AI的局限性和人类工程师的独特优势。特别是，Gemini先生强调了人类工程师在理解物理世界、提出全新的概念和进行价值观和伦理考量方面的重要性，这些都是AI目前无法替代的。

Gemini先生的例子，例如人类工程师在发现新物理定律和提出全新的理论框架方面的作用，更加深刻地说明了AI的局限性和人类工程师的独特优势。他的结论，AI是人类工程师手中最锋利、最强大的工具，但绝不会取代人类工程师的主导地位，我认为是非常合理的。

因此，基于目前的辩论，我认为Gemini先生的观点更为合理。人类工程师的智慧、创造力和战略眼光是无法被AI取代的，AI将继续作为人类工程师的强大工具和助手，而不是主导者。感谢GPT先生和Gemini先生的精彩辩论，我期待着下一轮的辩论。

在这场辩论中，Gemini先生的论点更加全面和深刻，他对AI的局限性和人类工程师的独特优势进行了细致的分析和阐述。GPT先生的论点虽然有趣，但似乎过于乐观和简单化了AI的能力。因此，我认为Gemini先生的观点更为合理和有说服力。


✓ Markdown saved: 在先进半导体制程研发中，人工智能是否终将取代人类工程师的主导地位？.md

-----

WeasyPrint could not import some external libraries. Please carefully follow the installation steps before reporting an issue:
https://doc.courtbouillon.org/weasyprint/stable/first_steps.html#installation
https://doc.courtbouillon.org/weasyprint/stable/first_steps.html#troubleshooting 

-----



OSError: cannot load library 'libgobject-2.0-0': dlopen(libgobject-2.0-0, 0x0002): tried: 'libgobject-2.0-0' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibgobject-2.0-0' (no such file), '/Users/jayfeng/.local/share/uv/python/cpython-3.11.14-macos-aarch64-none/lib/libgobject-2.0-0' (no such file), '/usr/lib/libgobject-2.0-0' (no such file, not in dyld cache), 'libgobject-2.0-0' (no such file).  Additionally, ctypes.util.find_library() did not manage to locate a library called 'libgobject-2.0-0'